In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import sys
sys.path.insert(0,'src/')

from collections import namedtuple

import torch 
from torch.utils.data import Dataset, DataLoader, random_split
import joblib
from torch import nn 
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd

from reward_evaluation import RewardComparision
from architecture.reward import StateDataset, EpisodeDataset, ImbalancedDatasetSampler, RewardModel
from architecture.language_model import LanguageModel
from simulator.description_embedder import Description_embedder
from simulator.Environment import preprocess_raw_observation
from simulator.Items import ITEM_TYPE
from config import generate_params

params = generate_params(save_path=False)

In [4]:
# EpisodeRecord = namedtuple('EpisodeRecord', ('initial_state', 'final_state', 'instruction', 'reward'))
# episode_path = 'results/episodes_records.jbl'
# episodes = joblib.load(episode_path)

StateRecord = namedtuple('StateRecord', ('state', 'instruction', 'reward'))
EpisodeRecord = namedtuple('EpisodeRecord', ('initial_state', 'final_state', 'instruction', 'reward'))

state_path = 'results/state_records3.jbl'
train_episode = 'results/episodes_records4.jbl'
test_episode = 'results/episodes_records3.jbl'

In [5]:
description_embedder = Description_embedder(**params['env_params']['description_embedder_params'])

item_type_embedder = OneHotEncoder(sparse=False)
item_type_embedder.fit(np.array(ITEM_TYPE).reshape(-1, 1))

from functools import partial
transformer = partial(preprocess_raw_observation, description_embedder=description_embedder, item_type_embedder=item_type_embedder, raw_state_size=3, 
                      pytorch=True, device=params['device'])


# dts = StateDataset.from_files(state_path, raw_state_transformer=transformer, max_size=5000)
# dts = EpisodeDataset.from_files(episode_path, raw_state_transformer=transformer)
# dts_loader = DataLoader(dts, batch_size=len(dts))
# train_dts, test_dts = dts.split(train_test_ratio = 0.85)

train_dts = EpisodeDataset.from_files(train_episode, raw_state_transformer=transformer)
test_dts = EpisodeDataset.from_files(test_episode, raw_state_transformer=transformer, max_size=5000)

sampler = ImbalancedDatasetSampler(train_dts, num_samples=20000, pos_weight=0.2)
# train_loader = DataLoader(dataset=train_dts, sampler=sampler, batch_size=128)
test_loader = DataLoader(test_dts, batch_size=len(test_dts))
test_batch = next(iter(test_loader))

In [109]:
positive_count = [len(v) for v in dts.positive_record.values()]
negative_count = [len(v) for v in dts.negative_record.values()]
df = pd.DataFrame([positive_count, negative_count]).T
df.index = list(dts.positive_record)
df.columns = columns = ['pos', 'neg']
df_overall = pd.DataFrame(df.sum()).T
df_overall.index = ['overall']
df = pd.concat([df, df_overall])
df['%pos'] = df.pos / (df.pos + df.neg)
df

,pos,neg,%pos
You increased the luminosity of first light bulb,4995,8603,0.367333
You turned off the first plug,4905,8766,0.358789
You made the light of first light bulb warmer,4292,8980,0.323388
You set the color of first light bulb to orange,247,9818,0.024540
The luminosity of first light bulb is now average,473,9816,0.045971
You set the color of first light bulb to purple,217,10073,0.021088
You decreased the luminosity of first light bulb,2047,9516,0.177030
The luminosity of first light bulb is now high,2070,9527,0.178494
The luminosity of first light bulb is now low,440,9958,0.042316
You made the light of first light bulb colder,4356,8881,0.329078


In [106]:
df_overall = pd.DataFrame(df.sum()).T
df.index = ['overall']
pd.concat([df, pd.DataFrame(df.sum()).T])

,pos,neg,%pos
You increased the luminosity of first light bulb,4995.0,8603.0,0.367333
You turned off the first plug,4905.0,8766.0,0.358789
You made the light of first light bulb warmer,4292.0,8980.0,0.323388
You set the color of first light bulb to orange,247.0,9818.0,0.024540
The luminosity of first light bulb is now average,473.0,9816.0,0.045971
You set the color of first light bulb to purple,217.0,10073.0,0.021088
You decreased the luminosity of first light bulb,2047.0,9516.0,0.177030
The luminosity of first light bulb is now high,2070.0,9527.0,0.178494
The luminosity of first light bulb is now low,440.0,9958.0,0.042316
You made the light of first light bulb colder,4356.0,8881.0,0.329078


In [7]:
language_model = LanguageModel(**params['language_model_params'])
reward_function = LearnedReward(context_model=params['model_params']['context_model'], language_model=language_model, reward_params=params['reward_model_params'])
reward_function.to(params['device'])

from torch import optim
loss_func = nn.BCELoss()
optimizer = optim.Adam(reward_function.parameters())

In [19]:
from tqdm.notebook import tqdm
from torch.nn.utils import clip_grad_norm_

for epoch in range(100):
    for i, batch in tqdm(enumerate(train_loader)):
#         print(batch['reward'].float().mean())
        optimizer.zero_grad()
        reward = reward_function(state=batch['state'], instructions=batch['instruction']).view(-1)
        loss = loss_func(reward, batch['reward'].float().to(params['device']))
        print(f'{epoch} {i} loss: {loss.item()}')
        loss.backward()
#         clip_grad_norm_(reward_function.parameters(), 1)
        optimizer.step()
        
    

0 0 loss: 0.05711089447140694
0 1 loss: 0.0653453916311264
0 2 loss: 0.061978090554475784
0 3 loss: 0.053842462599277496
0 4 loss: 0.074861079454422
0 5 loss: 0.07310312986373901
0 6 loss: 0.0656624585390091
0 7 loss: 0.08667128533124924
0 8 loss: 0.07506480067968369
0 9 loss: 0.1087091863155365
0 10 loss: 0.038395702838897705
0 11 loss: 0.04237275943160057
0 12 loss: 0.04104701429605484
0 13 loss: 0.07748584449291229
0 14 loss: 0.07600108534097672
0 15 loss: 0.1359160989522934
0 16 loss: 0.053344376385211945
0 17 loss: 0.04987293854355812
0 18 loss: 0.07306171953678131
0 19 loss: 0.064381904900074
0 20 loss: 0.05508366972208023
0 21 loss: 0.10977333039045334
0 22 loss: 0.056368082761764526
0 23 loss: 0.04375394433736801
0 24 loss: 0.04222206771373749
0 25 loss: 0.031051531434059143
0 26 loss: 0.0839010626077652
0 27 loss: 0.06733690202236176
0 28 loss: 0.07599324733018875
0 29 loss: 0.06937453150749207
0 30 loss: 0.07159052789211273
0 31 loss: 0.057998619973659515
0 32 loss: 0.0359527

1 0 loss: 0.09296280145645142
1 1 loss: 0.08203186094760895
1 2 loss: 0.024309052154421806
1 3 loss: 0.0258091501891613
1 4 loss: 0.06984315067529678
1 5 loss: 0.07733271270990372
1 6 loss: 0.0597383975982666
1 7 loss: 0.09382972121238708
1 8 loss: 0.12280167639255524
1 9 loss: 0.014874305576086044
1 10 loss: 0.10073670744895935
1 11 loss: 0.015416206791996956
1 12 loss: 0.058806151151657104
1 13 loss: 0.06679844856262207
1 14 loss: 0.07519792020320892
1 15 loss: 0.0751529112458229
1 16 loss: 0.05818849056959152
1 17 loss: 0.07148868590593338
1 18 loss: 0.12956681847572327
1 19 loss: 0.06178022921085358
1 20 loss: 0.024018920958042145
1 21 loss: 0.053804103285074234
1 22 loss: 0.03466997295618057
1 23 loss: 0.02485303208231926
1 24 loss: 0.028028912842273712
1 25 loss: 0.04680359736084938
1 26 loss: 0.06093886122107506
1 27 loss: 0.06623439490795135
1 28 loss: 0.06572945415973663
1 29 loss: 0.04558172821998596
1 30 loss: 0.01725270226597786
1 31 loss: 0.03375369310379028
1 32 loss: 0.0

2 0 loss: 0.03337174654006958
2 1 loss: 0.04682227596640587
2 2 loss: 0.03618162125349045
2 3 loss: 0.032740119844675064
2 4 loss: 0.04480356350541115
2 5 loss: 0.042017433792352676
2 6 loss: 0.041028186678886414
2 7 loss: 0.03448895364999771
2 8 loss: 0.04038792848587036
2 9 loss: 0.017448697239160538
2 10 loss: 0.04377584159374237
2 11 loss: 0.04479197412729263
2 12 loss: 0.019688330590724945
2 13 loss: 0.0472196526825428
2 14 loss: 0.061010461300611496
2 15 loss: 0.03218396008014679
2 16 loss: 0.034729816019535065
2 17 loss: 0.00994601659476757
2 18 loss: 0.06617330014705658
2 19 loss: 0.04089813306927681
2 20 loss: 0.03483204171061516
2 21 loss: 0.06146489828824997
2 22 loss: 0.06752562522888184
2 23 loss: 0.045651137828826904
2 24 loss: 0.050140395760536194
2 25 loss: 0.030463771894574165
2 26 loss: 0.04521387815475464
2 27 loss: 0.07195288687944412
2 28 loss: 0.06800715625286102
2 29 loss: 0.07367274165153503
2 30 loss: 0.04514716938138008
2 31 loss: 0.046817611902952194
2 32 los

3 0 loss: 0.08428271114826202
3 1 loss: 0.027456112205982208
3 2 loss: 0.09462219476699829
3 3 loss: 0.01566447876393795
3 4 loss: 0.04718851298093796
3 5 loss: 0.027409913018345833
3 6 loss: 0.08315487951040268
3 7 loss: 0.05002908036112785
3 8 loss: 0.04749758541584015
3 9 loss: 0.06074657291173935
3 10 loss: 0.05208876356482506
3 11 loss: 0.007057345937937498
3 12 loss: 0.04072653502225876
3 13 loss: 0.016785304993391037
3 14 loss: 0.02056990936398506
3 15 loss: 0.03867800161242485
3 16 loss: 0.015535256825387478
3 17 loss: 0.04713313281536102
3 18 loss: 0.009257483296096325
3 19 loss: 0.03119269385933876
3 20 loss: 0.013982422649860382
3 21 loss: 0.09666761755943298
3 22 loss: 0.015513461083173752
3 23 loss: 0.030811738222837448
3 24 loss: 0.05484171211719513
3 25 loss: 0.014986051246523857
3 26 loss: 0.02442403882741928
3 27 loss: 0.015021842904388905
3 28 loss: 0.017285803332924843
3 29 loss: 0.06836239248514175
3 30 loss: 0.02588675729930401
3 31 loss: 0.0904235988855362
3 32 lo

4 0 loss: 0.04151609167456627
4 1 loss: 0.016714025288820267
4 2 loss: 0.03581998497247696
4 3 loss: 0.051441218703985214
4 4 loss: 0.03564375266432762
4 5 loss: 0.028141947463154793
4 6 loss: 0.017577195540070534
4 7 loss: 0.10522892326116562
4 8 loss: 0.01462179608643055
4 9 loss: 0.013045523315668106
4 10 loss: 0.024240415543317795
4 11 loss: 0.05541915446519852
4 12 loss: 0.047104496508836746
4 13 loss: 0.054926276206970215
4 14 loss: 0.03547455742955208
4 15 loss: 0.01032279059290886
4 16 loss: 0.046620212495326996
4 17 loss: 0.0688522532582283
4 18 loss: 0.02677685208618641
4 19 loss: 0.0452757328748703
4 20 loss: 0.05904532968997955
4 21 loss: 0.005617184564471245
4 22 loss: 0.02785094641149044
4 23 loss: 0.017500318586826324
4 24 loss: 0.029569869861006737
4 25 loss: 0.07705800235271454
4 26 loss: 0.017062749713659286
4 27 loss: 0.06715104728937149
4 28 loss: 0.039408788084983826
4 29 loss: 0.0680784061551094
4 30 loss: 0.014094913378357887
4 31 loss: 0.008426228538155556
4 32 

...

27 0 loss: 0.005566351115703583
27 1 loss: 0.023680349811911583
27 2 loss: 0.0031151392031461
27 3 loss: 0.013040799647569656
27 4 loss: 0.0019548030104488134
27 5 loss: 0.03125078231096268
27 6 loss: 0.00844897236675024
27 7 loss: 0.03688947483897209
27 8 loss: 0.027911055833101273
27 9 loss: 0.001968958182260394
27 10 loss: 0.0019298868719488382
27 11 loss: 0.004389924928545952
27 12 loss: 0.0014345443341881037
27 13 loss: 0.002289121737703681
27 14 loss: 0.05951056629419327
27 15 loss: 0.019980955868959427
27 16 loss: 0.0012398614780977368
27 17 loss: 0.0021541444584727287
27 18 loss: 0.0018949367804452777
27 19 loss: 0.007975883781909943
27 20 loss: 0.013009521178901196
27 21 loss: 0.0030793650075793266
27 22 loss: 0.012531698681414127
27 23 loss: 0.0012855236418545246
27 24 loss: 0.006622892804443836
27 25 loss: 0.008938828483223915
27 26 loss: 0.00681835412979126
27 27 loss: 0.005372252780944109
27 28 loss: 0.002596631646156311
27 29 loss: 0.006902789697051048
27 30 loss: 0.00159

28 0 loss: 0.006476474925875664
28 1 loss: 0.0010491901775822043
28 2 loss: 0.00425384659320116
28 3 loss: 0.03188919275999069
28 4 loss: 0.0016144505934789777
28 5 loss: 0.028713349252939224
28 6 loss: 0.0017805984243750572
28 7 loss: 0.0013502074871212244
28 8 loss: 0.03345917537808418
28 9 loss: 0.0012002987787127495
28 10 loss: 0.018260009586811066
28 11 loss: 0.0069276620633900166
28 12 loss: 0.0033709038980305195
28 13 loss: 0.0040510669350624084
28 14 loss: 0.0018484113970771432
28 15 loss: 0.006472433917224407
28 16 loss: 0.04778840392827988
28 17 loss: 0.003050491213798523
28 18 loss: 0.0010416615987196565
28 19 loss: 0.0019612612668424845
28 20 loss: 0.003379985922947526
28 21 loss: 0.0016920609632506967
28 22 loss: 0.0015923792961984873
28 23 loss: 0.001844647340476513
28 24 loss: 0.018280593678355217
28 25 loss: 0.058878056704998016
28 26 loss: 0.0010222892742604017
28 27 loss: 0.005092845298349857
28 28 loss: 0.0011546118184924126
28 29 loss: 0.017735859379172325
28 30 los

29 0 loss: 0.00769854336977005
29 1 loss: 0.002317601116374135
29 2 loss: 0.0012314605992287397
29 3 loss: 0.0037064754869788885
29 4 loss: 0.0020807734690606594
29 5 loss: 0.008603353053331375
29 6 loss: 0.004566385876387358
29 7 loss: 0.01874910667538643
29 8 loss: 0.001350344973616302
29 9 loss: 0.0009574509458616376
29 10 loss: 0.0008129746420308948
29 11 loss: 0.0029295908752828836
29 12 loss: 0.0006618694169446826
29 13 loss: 0.0015668454580008984
29 14 loss: 0.0008722062921151519
29 15 loss: 0.0017480660462751985
29 16 loss: 0.07476504892110825
29 17 loss: 0.0010003719944506884
29 18 loss: 0.027779672294855118
29 19 loss: 0.000731695385184139
29 20 loss: 0.0004316282575018704
29 21 loss: 0.007370827719569206
29 22 loss: 0.0007251653005369008
29 23 loss: 0.001799687510356307
29 24 loss: 0.03543168678879738
29 25 loss: 0.01805134303867817
29 26 loss: 0.04131472483277321
29 27 loss: 0.02505001612007618
29 28 loss: 0.0036299657076597214
29 29 loss: 0.019296659156680107
29 30 loss: 0

30 0 loss: 0.0005184750771149993
30 1 loss: 0.0011795840691775084
30 2 loss: 0.004821775946766138
30 3 loss: 0.004598965402692556
30 4 loss: 0.0003000882570631802
30 5 loss: 0.010442828759551048
30 6 loss: 0.0005409301957115531
30 7 loss: 0.0337037555873394
30 8 loss: 0.05583368241786957
30 9 loss: 0.002017894061282277
30 10 loss: 0.0009168143151327968
30 11 loss: 0.00017169208149425685
30 12 loss: 0.00045145017793402076
30 13 loss: 0.005361748859286308
30 14 loss: 0.01590864732861519
30 15 loss: 0.008459744974970818
30 16 loss: 0.01094420813024044
30 17 loss: 0.0032454298343509436
30 18 loss: 0.0054534487426280975
30 19 loss: 0.002202940173447132
30 20 loss: 0.022141309455037117
30 21 loss: 0.0031852282118052244
30 22 loss: 0.016311032697558403
30 23 loss: 0.001965909032151103
30 24 loss: 0.0014590061036869884
30 25 loss: 0.0015257211634889245
30 26 loss: 0.004495800007134676
30 27 loss: 0.003442243905737996
30 28 loss: 0.010601327754557133
30 29 loss: 0.0008137632394209504
30 30 loss

KeyboardInterrupt: 

In [70]:
len(train_dts)

170000

In [16]:
test_batch = next(iter(test_loader))

In [20]:
from sklearn.metrics import recall_score, f1_score, precision_score, accuracy_score

def compute_stats(g):
    count = len(g)
    pred_1 = g.pred.sum()
    pred_0 = (1 - g.pred).sum()
    true_1 = g.true.sum()
    true_0 = (1 - g.true).sum()
    stats = [count, true_0, pred_0, true_1, pred_1]
    stats_name = ['count', 'true_0', 'pred_0', 'true_1', 'pred_1']
    result = pd.DataFrame(stats).T
    result.columns = stats_name
    return result

def compute_classification_metrics(g):
    accuracy = accuracy_score(g.true, g.pred)
    precision = precision_score(g.true, g.pred)
    recall = recall_score(g.true, g.pred)
    f1 = f1_score(g.true, g.pred)
    scores = [accuracy, precision, recall, f1]
    score_name = ['accuracy', 'precision', 'recall', 'f1_score']
    result = pd.DataFrame(scores).T
    result.columns = score_name
    return result
    
def compute_metrics(g, data_stats=False):
    df_score = compute_classification_metrics(g)
    df_stats = compute_stats(g) if data_stats else pd.DataFrame()
    result = pd.concat([df_stats, df_score], axis=1)
    return result

test_reward = reward_function(state=test_batch['state'], instructions=test_batch['instruction'])#.view(-1)
torch.cat([test_reward.cpu(), test_batch['reward'].float().view(-1, 1)], dim=1)
pred_true_tensor = torch.cat([(test_reward >0.5).float().cpu(), test_batch['reward'].float().view(-1, 1)], dim=1) 
df = pd.DataFrame(pred_true_tensor.detach().numpy(), columns=['pred', 'true'])
df['instruction'] = test_batch['instruction']
stats = df.groupby('instruction').apply(compute_metrics, data_stats=True)
stats

/home/nicolas/miniconda3/envs/imagineIOT/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nicolas/miniconda3/envs/imagineIOT/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nicolas/miniconda3/envs/imagineIOT/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1514: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


,,count,true_0,pred_0,true_1,pred_1,accuracy,precision,recall,f1_score
instruction,,,,,,,,,,
The luminosity of first light bulb is now average,0,247.0,245.0,245.0,2.0,2.0,1.000000,1.000000,1.0,1.000000
The luminosity of first light bulb is now high,0,219.0,198.0,195.0,21.0,24.0,0.986301,0.875000,1.0,0.933333
The luminosity of first light bulb is now low,0,263.0,258.0,258.0,5.0,5.0,1.000000,1.000000,1.0,1.000000
The luminosity of first light bulb is now very high,0,251.0,243.0,242.0,8.0,9.0,0.996016,0.888889,1.0,0.941176
The luminosity of first light bulb is now very low,0,229.0,220.0,220.0,9.0,9.0,1.000000,1.000000,1.0,1.000000
You decreased the luminosity of first light bulb,0,269.0,255.0,254.0,14.0,15.0,0.996283,0.933333,1.0,0.965517
You increased the luminosity of first light bulb,0,246.0,220.0,220.0,26.0,26.0,1.000000,1.000000,1.0,1.000000
You made the light of first light bulb colder,0,242.0,215.0,215.0,27.0,27.0,1.000000,1.000000,1.0,1.000000
You made the light of first light bulb warmer,0,243.0,206.0,206.0,37.0,37.0,1.000000,1.000000,1.0,1.000000


In [21]:
stats.f1_score.mean()

0.930035540299232

In [83]:
a
compute_classification_metrics(df)

,accuracy,precision,recall,f1_score
0,0.9726,0.683014,0.718773,0.700437


In [89]:
positive_count = [len(v) for v in dts.positive_record.values()]
negative_count = [len(v) for v in dts.negative_record.values()]
df = pd.DataFrame([positive_count, negative_count]).T
df['instruction'] = list(dts.positive_record)
df

,0,1,instruction
0,255,9745,You turned off the first light bulb
1,248,9752,The luminosity of first light bulb is now very...
2,62,9938,You set the color of first light bulb to orange
3,37,9963,You set the color of first light bulb to green
4,1076,8924,You made the light of first light bulb warmer
5,826,9174,You turned on the first light bulb
6,1102,8898,You made the light of first light bulb colder
7,516,9484,The luminosity of first light bulb is now high
8,292,9708,The luminosity of first light bulb is now very...
9,113,9887,The luminosity of first light bulb is now low


In [14]:
dts_batch = next(iter(dts_loader))

In [53]:
def compute_stats(g):
    count = len(g)
    pos = g.reward.sum()
    neg = (1 - g.reward).sum()
    stats = [count, neg, pos]
    stats_name = ['count', 'neg', 'pos']
    result = pd.DataFrame(stats).T
    result.columns = stats_name
    return result

df_dts = pd.DataFrame(dts_batch['reward'].numpy(), columns=['reward'])
df_dts['instruction'] = dts_batch['instruction']
df_dts.groupby('instruction').apply(compute_stats)

,,count,neg,pos
instruction,,,,
The luminosity of first light bulb is now average,0,5000,4934,66
The luminosity of first light bulb is now high,0,5000,4791,209
The luminosity of first light bulb is now low,0,5000,4946,54
The luminosity of first light bulb is now very high,0,5000,4856,144
The luminosity of first light bulb is now very low,0,5000,4894,106
You decreased the luminosity of first light bulb,0,5000,4757,243
You increased the luminosity of first light bulb,0,5000,4443,557
You made the light of first light bulb colder,0,5000,4441,559
You made the light of first light bulb warmer,0,5000,4431,569


## Compare reward function 

In [6]:
class Net(nn.Module):
    def __init__(self, n_inputs):
        super(Net, self).__init__()
        self.shared_encoding = nn.Sequential(nn.Linear(1, 100),
                                             nn.ReLU(),
                                             nn.Linear(100, n_inputs))
        self.out_layer = nn.Sequential(nn.Linear(n_inputs, 100),
                                       nn.ReLU(),
                                       nn.Linear(100, 1),
                                       nn.Sigmoid())

    def forward(self, x):
        latent = self.shared_encoding(x.unsqueeze(dim=2))
        latent = latent.sum(dim=1)
        out = self.out_layer(latent)
        return out

In [20]:
language_model = LanguageModel(**params['language_model_params'])
reward = RewardModel(context_model=params['model_params']['context_model'], language_model=language_model,
                     reward_params=params['reward_params']['net_params'])

reward_net_params = params['reward_params']['net_params'].copy()
reward_net_params.update(aggregate='diff_or',
                         scaler_layer_params=dict(hidden1_out=256, latent_out=1, last_activation='sigmoid')
                         )
prelearned_or_reward = RewardModel(context_model=params['model_params']['context_model'],
                                   language_model=language_model,
                                   reward_params=reward_net_params)

prelearned_or_reward = RewardModel(context_model=params['model_params']['context_model'],
                                   language_model=language_model,
                                   reward_params=reward_net_params)
reward_list= [reward, prelearned_or_reward]
evaluator = RewardComparision(reward_list, fit_params=params['reward_params']['fit_params'],
                              device=params['device'])

In [ ]:
metrics = evaluator.run(train_dts, test_batch, data_stats=False)

Epoch: 0: loss: [0.49062734842300415, 0.5331212282180786]
Epoch: 1: loss: [0.42949768900871277, 0.5039872527122498]
Epoch: 2: loss: [0.38533565402030945, 0.4924776554107666]
Epoch: 3: loss: [0.3754216730594635, 0.5010824203491211]
Epoch: 4: loss: [0.36204811930656433, 0.5044633150100708]
Epoch: 5: loss: [0.31432783603668213, 0.4790242612361908]
Epoch: 6: loss: [0.30259013175964355, 0.48583856225013733]
Epoch: 7: loss: [0.2990313172340393, 0.48466721177101135]
Epoch: 8: loss: [0.27609536051750183, 0.48269522190093994]
Epoch: 9: loss: [0.2726542353630066, 0.4854130446910858]
Epoch: 10: loss: [0.25814196467399597, 0.4773463010787964]
Epoch: 11: loss: [0.2654891312122345, 0.4790959358215332]
Epoch: 12: loss: [0.2476426512002945, 0.49161896109580994]
Epoch: 13: loss: [0.23864726722240448, 0.47595205903053284]
Epoch: 14: loss: [0.24070262908935547, 0.48695698380470276]
Epoch: 15: loss: [0.23039725422859192, 0.48434388637542725]
Epoch: 16: loss: [0.23314276337623596, 0.47656306624412537]
Epoc

In [30]:
metrics[0].tail(21)

,index,precision,recall,f1_score,epoch
2079,The luminosity of first light bulb is now average,0.100000,0.666667,0.173913,99
2080,The luminosity of first light bulb is now high,0.153846,1.000000,0.266667,99
2081,The luminosity of first light bulb is now low,0.078947,1.000000,0.146341,99
2082,The luminosity of first light bulb is now very...,0.291667,0.875000,0.437500,99
2083,The luminosity of first light bulb is now very...,0.205128,1.000000,0.340426,99
2084,You decreased the luminosity of first light bulb,0.333333,0.625000,0.434783,99
2085,You increased the luminosity of first light bulb,0.740741,0.800000,0.769231,99
2086,You made the light of first light bulb colder,0.000000,0.000000,0.000000,99
2087,You made the light of first light bulb warmer,0.000000,0.000000,0.000000,99
2088,You set the color of first light bulb to blue,0.000000,0.000000,0.000000,99


In [31]:
metrics[1].tail(21)

,index,precision,recall,f1_score,epoch
2079,The luminosity of first light bulb is now average,0.000000,0.000000,0.000000,99
2080,The luminosity of first light bulb is now high,0.200000,0.500000,0.285714,99
2081,The luminosity of first light bulb is now low,0.000000,0.000000,0.000000,99
2082,The luminosity of first light bulb is now very...,0.666667,0.500000,0.571429,99
2083,The luminosity of first light bulb is now very...,0.000000,0.000000,0.000000,99
2084,You decreased the luminosity of first light bulb,0.000000,0.000000,0.000000,99
2085,You increased the luminosity of first light bulb,1.000000,0.240000,0.387097,99
2086,You made the light of first light bulb colder,0.000000,0.000000,0.000000,99
2087,You made the light of first light bulb warmer,0.000000,0.000000,0.000000,99
2088,You set the color of first light bulb to blue,0.000000,0.000000,0.000000,99
